In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd
import geopandas as gpd
import timeit
import geopy.distance
import numpy as np
import math
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPoint, Polygon, LineString
import shapely.wkt
import random
from pyproj import Proj, transform #Convertion EPSG2154 to WGS84

from datetime import datetime

In [2]:
#<vehicle id="taxi0" start_link="25484" t_0="21600.0" t_1="86400.0" capacity="1"/>

In [3]:
### Loading df stations
df_taxis_stations = pd.read_csv("/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis/taxis_stations_with_net_link.csv", sep=",")

In [4]:
df_taxis_stations

,nom,adresse,commune,nbemplacem,telephone,separateur,abri,panneau,diodes,totem,gid,geometry,link,link_id
0,TAX901,Rue de la Pépinière Royale,Lyon 9 ème,2.0,Non,Autre,Non,Oui,Non,Non,1,POINT (840278.5059883833 6520207.191972129),LINESTRING (840301.1588493029 6520331.08429506...,51451
1,TAXSFO01,Rue Charles Plasse,SAINT-FONS,3.0,Non,Bordure granit,Non,Oui,Non,Non,2,POINT (844187.3004137476 6513847.712222898),LINESTRING (844226.0349661598 6513836.70919504...,24884
2,TAX312,Rue Aimé Collomb,Lyon 3 ème,4.0,Non,Peinture,Non,Oui,Non,Non,3,POINT (843209.60109396 6519131.609318716),LINESTRING (843182.5186245841 6519147.71246535...,70751
3,TAXVIL03,"158, Rue Léon Blum",VILLEURBANNE,4.0,Non,Bordure granit,Non,Oui,Non,Oui,4,POINT (848171.8562919963 6519672.127143201),LINESTRING (848108.9061020506 6519670.51450241...,44208
4,TAXRIL02,"79b, Avenue de l'Europe",RILLIEUX-LA-PAPE,3.0,Non,Bordure granit,Non,Oui,Non,Non,5,POINT (847709.1394807356 6526028.936303516),LINESTRING (847765.5656705137 6526052.80299619...,10216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TAXNEU01,Quai Pasteur,NEUVILLE-SUR-SAONE,4.0,Oui,Peinture,Oui,Oui,Non,Non,106,POINT (842577.9298042508 6532441.7974093305),LINESTRING (842578.6911077248 6532469.04649189...,4042
106,TAX902,Rue Roger Salengro,Lyon 9 ème,5.0,Non,Bordure granit,Non,Oui,Non,Oui,107,POINT (840161.0834162626 6521569.810913701),LINESTRING (840182.0339989121 6521553.87081078...,48959
107,TAX703,"16, Place Docteurs Charles et Christophe Mérieux",Lyon 7 ème,10.0,Non,Bordure granit,Non,Oui,Non,Oui,108,POINT (841987.0938321726 6516362.568314445),LINESTRING (841991.3932604335 6516438.16012393...,4259
108,TAX904,"Face 8, Rue de la Navigation",Lyon 9 ème,4.0,Non,Bordure granit,Non,Oui,Non,Oui,109,POINT (840806.8584226499 6522555.1337179635),LINESTRING (840824.4486077658 6522547.24456913...,13512


In [5]:
df_taxis_stations["commune"].unique()

array(['Lyon 9 ème', 'SAINT-FONS', 'Lyon 3 ème', 'VILLEURBANNE',
       'RILLIEUX-LA-PAPE', 'OULLINS', 'CHARBONNIERES-LES-BAINS', 'CORBAS',
       'LIMONEST', 'Lyon 2 ème', 'BRON', 'CHASSIEU', 'DARDILLY',
       'LA TOUR-DE-SALVAGNY', 'MIONS', 'ROCHETAILLEE-SUR-SAONE',
       "SAINT-DIDIER-AU-MONT-D'OR", "SAINT-GERMAIN-AU-MONT-D'OR",
       'LISSIEU', 'GIVORS', 'VERNAISON', 'Lyon 8 ème', 'VAULX-EN-VELIN',
       'FONTAINES-SUR-SAONE', 'FLEURIEU-SUR-SAONE', 'SAINTE-FOY-LES-LYON',
       'MEYZIEU', 'Lyon 6 ème', 'GENAY', "CHAMPAGNE-AU-MONT-D'OR",
       'Lyon 7 ème', 'ECULLY', 'SAINT-PRIEST', 'TASSIN-LA-DEMI-LUNE',
       'VENISSIEUX', 'Lyon 4 ème', 'Lyon 5 ème', 'CALUIRE-ET-CUIRE',
       'Lyon 1 er', 'NEUVILLE-SUR-SAONE'], dtype=object)

In [6]:
### Select only station from the define urban aera
urban_taxis = ["Lyon 9 ème", "Lyon 3 ème", "VILLEURBANNE", "Lyon 2 ème", "BRON", "Lyon 8 ème", "VAULX-EN-VELIN", "Lyon 6 ème", "Lyon 7 ème", "Lyon 4 ème", "Lyon 5 ème", "CALUIRE-ET-CUIRE", "Lyon 1 er"]

In [7]:
df_taxis_stations = df_taxis_stations[df_taxis_stations["commune"].isin(urban_taxis)]

In [8]:
df_taxis_stations["commune"].unique()

array(['Lyon 9 ème', 'Lyon 3 ème', 'VILLEURBANNE', 'Lyon 2 ème', 'BRON',
       'Lyon 8 ème', 'VAULX-EN-VELIN', 'Lyon 6 ème', 'Lyon 7 ème',
       'Lyon 4 ème', 'Lyon 5 ème', 'CALUIRE-ET-CUIRE', 'Lyon 1 er'],
      dtype=object)

In [9]:
#First case
#number_of_taxis = int(12779 * 10 /100) #10% de la flotte de VP. See population post-processing
#df_taxis = df_taxis_stations.sample(n=number_of_taxis, weights = df_taxis_stations.nbemplacem, replace=True)

In [10]:
int(12779 * 10 /100)

1277

In [11]:
#Second case 
#number_of_taxis = int(12779 * 15 /100)#15% de la flotte de VP. See population post-processing
#df_taxis = df_taxis_stations.sample(n=number_of_taxis, weights = df_taxis_stations.nbemplacem, replace=True)#

In [12]:
int(12779 * 15 /100)

1916

In [13]:
#Third case 
#number_of_taxis = int(12779 * 20 /100)#20% de la flotte de VP. See population post-processing
#df_taxis = df_taxis_stations.sample(n=number_of_taxis, weights = df_taxis_stations.nbemplacem, replace=True)

In [14]:
int(12779 * 20 /100)

2555

In [15]:
#Four case 
number_of_taxis = int(12779 * 25 /100)#25% de la flotte de VP. See population post-processing
df_taxis = df_taxis_stations.sample(n=number_of_taxis, weights = df_taxis_stations.nbemplacem, replace=True)

In [16]:
int(12779 * 25 /100)

3194

In [17]:
df_taxis.head(20)

,nom,adresse,commune,nbemplacem,telephone,separateur,abri,panneau,diodes,totem,gid,geometry,link,link_id
54,TAX208,Place des Cordeliers,Lyon 2 ème,8.0,Non,Peinture,Non,Non,Non,Oui,55,POINT (842717.2470816755 6519879.5495876605),LINESTRING (842677.4276167355 6519864.82714292...,29452
43,TAX311,"107, Cours Albert Thomas",Lyon 3 ème,4.0,Non,Bordure granit,Non,Oui,Oui,Oui,44,POINT (845506.9613870343 6517935.068989062),LINESTRING (845535.3188506773 6517914.07612975...,58891
87,TAX601,Place Jules Ferry,Lyon 6 ème,9.0,Non,Peinture,Non,Oui,Non,Oui,88,POINT (844499.3067759846 6520239.563958139),LINESTRING (844488.4163281866 6520216.05758647...,56858
100,TAX101,"5, Rue Président Edouard Herriot",Lyon 1 er,3.0,Non,Peinture,Non,Oui,Non,Non,101,POINT (842562.133592683 6520213.25595944),LINESTRING (842557.581684675 6520183.020487789...,47802
94,TAX803,Rue du Professeur Beauvisage,Lyon 8 ème,5.0,Non,Bordure granit,Non,Oui,Non,Oui,95,POINT (845209.0083280387 6516111.450745643),LINESTRING (845161.5332749675 6516093.14339709...,74874
96,TAX606,"10, Quai Charles de Gaulle",Lyon 6 ème,13.0,Non,Peinture,Non,Oui,Non,Oui,97,POINT (844326.5890740652 6522370.617161019),LINESTRING (844084.9113001837 6522380.76402456...,37368
92,TAX310,"1, Place d'arsonval",Lyon 3 ème,30.0,Oui,Peinture,Non,Non,Non,Non,93,POINT (846093.8228270261 6517732.7448239345),LINESTRING (846068.6918961094 6517763.87753333...,35869
32,TAX211,"17, Quai Jules Courmont",Lyon 2 ème,4.0,Non,Bordure granit,Non,Oui,Non,Non,33,POINT (842809.0335830599 6519409.237762606),LINESTRING (842819.4946057377 6519413.85588296...,72337
62,TAX304,"12, Boulevard Marius Vivier-Merle",Lyon 3 ème,7.0,Non,Bordure granit,Non,Non,Oui,Oui,63,POINT (844379.2947014435 6519752.806141034),LINESTRING (844375.3452196538 6519774.36820653...,8642
87,TAX601,Place Jules Ferry,Lyon 6 ème,9.0,Non,Peinture,Non,Oui,Non,Oui,88,POINT (844499.3067759846 6520239.563958139),LINESTRING (844488.4163281866 6520216.05758647...,56858


In [18]:
df_taxis = df_taxis.reset_index()

In [19]:
df_taxis

,index,nom,adresse,commune,nbemplacem,telephone,separateur,abri,panneau,diodes,totem,gid,geometry,link,link_id
0,54,TAX208,Place des Cordeliers,Lyon 2 ème,8.0,Non,Peinture,Non,Non,Non,Oui,55,POINT (842717.2470816755 6519879.5495876605),LINESTRING (842677.4276167355 6519864.82714292...,29452
1,43,TAX311,"107, Cours Albert Thomas",Lyon 3 ème,4.0,Non,Bordure granit,Non,Oui,Oui,Oui,44,POINT (845506.9613870343 6517935.068989062),LINESTRING (845535.3188506773 6517914.07612975...,58891
2,87,TAX601,Place Jules Ferry,Lyon 6 ème,9.0,Non,Peinture,Non,Oui,Non,Oui,88,POINT (844499.3067759846 6520239.563958139),LINESTRING (844488.4163281866 6520216.05758647...,56858
3,100,TAX101,"5, Rue Président Edouard Herriot",Lyon 1 er,3.0,Non,Peinture,Non,Oui,Non,Non,101,POINT (842562.133592683 6520213.25595944),LINESTRING (842557.581684675 6520183.020487789...,47802
4,94,TAX803,Rue du Professeur Beauvisage,Lyon 8 ème,5.0,Non,Bordure granit,Non,Oui,Non,Oui,95,POINT (845209.0083280387 6516111.450745643),LINESTRING (845161.5332749675 6516093.14339709...,74874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,103,TAX704,"256, Rue Marcel Mérieux",Lyon 7 ème,6.0,Non,Peinture,Non,Oui,Non,Non,104,POINT (842350.1496289071 6515807.442239112),LINESTRING (842358.6375648695 6515780.61297324...,1193
3190,101,TAX306,"32, Boulevard Eugéne Deruelle",Lyon 3 ème,5.0,Non,Bordure granit,Non,Oui,Non,Oui,102,POINT (844245.6113201163 6519859.9643837605),LINESTRING (844194.7035225789 6519857.75257300...,80333
3191,43,TAX311,"107, Cours Albert Thomas",Lyon 3 ème,4.0,Non,Bordure granit,Non,Oui,Oui,Oui,44,POINT (845506.9613870343 6517935.068989062),LINESTRING (845535.3188506773 6517914.07612975...,58891
3192,9,TAX200,"1, Rue de la Barre",Lyon 2 ème,10.0,Non,Peinture,Non,Oui,Oui,Oui,10,POINT (842586.8586107425 6519248.292124154),LINESTRING (842657.7036430828 6519215.02326919...,75075


In [20]:
output_path = "/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis" 

In [21]:
df_taxis.to_csv("%s/df_taxis_25_p_cent_3194.csv" % output_path)

In [22]:
#### Scenario 2: SAV
### Taxi service time
operationStartTime = 0.0# Starting time = 00h00
operationEndTime = 108000# 30h until the simulation finish
# Capacity of the taxi
seats = 1

tab="	"
vehicle_file_content='<!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">\n<vehicles>\n'
file_name = "/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis/taxis_sce2_v2_25_p_cent_3194.xml"

#Consider all taxis stations
for i in range(len(df_taxis)):
    vh_id = df_taxis["nom"][i]
    start_link = df_taxis["link_id"][i]
    vehicle_file_content=vehicle_file_content+tab+'<vehicle id="'+str(vh_id)+str(i)+'"'+tab+'start_link="'+str(start_link)+'"'+tab+'t_0="'+str(operationStartTime)+'"'+tab+'t_1="'+str(operationEndTime)+'"'+tab+'capacity="'+str(seats)+'"'+'/>\n'    

##Close the file           
vehicle_file_content=vehicle_file_content+'</vehicles>'
while os.path.isfile(file_name):
    file_name=file_name[:len(file_name)-4]+"_copy.xml"
file=open(file_name,"w")
file.write(vehicle_file_content)
print(vehicle_file_content)
file.close()

<!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">
<vehicles>
	<vehicle id="TAX2080"	start_link="29452"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX3111"	start_link="58891"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX6012"	start_link="56858"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX1013"	start_link="47802"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX8034"	start_link="74874"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX6065"	start_link="37368"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX3106"	start_link="35869"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX2117"	start_link="72337"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX3048"	start_link="8642"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX6019"	start_link="56858"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX30310"	start_link="57735"	t_0="0.0"	t_1="108000"	capacity="1"/>
	<vehicle id="TAX30611"	start_l

In [23]:
#### Scenario 3: PSAV
### Taxi service time
operationStartTime = 0.0# Starting time = 00h00
operationEndTime = 108000# 30h until the simulation finish
# Capacity of the taxi
seats = 4

tab="	"
vehicle_file_content='<!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">\n<vehicles>\n'
file_name = "/media/aodiallo/Data/MATSIM/Simulation/robotaxi/input/taxis/taxis_sce3_v2_25_p_cent_3194.xml"

#Consider all taxis stations
for i in range(len(df_taxis)):
    vh_id = df_taxis["nom"][i]
    start_link = df_taxis["link_id"][i]
    vehicle_file_content=vehicle_file_content+tab+'<vehicle id="'+str(vh_id)+str(i)+'"'+tab+'start_link="'+str(start_link)+'"'+tab+'t_0="'+str(operationStartTime)+'"'+tab+'t_1="'+str(operationEndTime)+'"'+tab+'capacity="'+str(seats)+'"'+'/>\n'    

##Close the file           
vehicle_file_content=vehicle_file_content+'</vehicles>'
while os.path.isfile(file_name):
    file_name=file_name[:len(file_name)-4]+"_copy.xml"
file=open(file_name,"w")
file.write(vehicle_file_content)
print(vehicle_file_content)
file.close()

<!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">
<vehicles>
	<vehicle id="TAX2080"	start_link="29452"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX3111"	start_link="58891"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX6012"	start_link="56858"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX1013"	start_link="47802"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX8034"	start_link="74874"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX6065"	start_link="37368"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX3106"	start_link="35869"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX2117"	start_link="72337"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX3048"	start_link="8642"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX6019"	start_link="56858"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX30310"	start_link="57735"	t_0="0.0"	t_1="108000"	capacity="4"/>
	<vehicle id="TAX30611"	start_l